# 神经网络的学习
- 从训练数据集中获得最优参数的过程，记神经网络的反向传播过程
- 深度学习是一种新的编程方法，摆脱了以往以人为中心的思想，所有的东西都让数据发声
## 传统算法&机器学习&深度学习
- 传统算法：人能想到的算法,然后编写算法规则，即把假设空间写死
- 机器学习：人能想到的特征，提取特征向量，机器学习（SVM/KNN）
- 深度学习：数据->答案
- 所以也称机器学习是一种端到端的学习方式
## 以识别手写数字为例说明三种算法的思路
### 传统算法
- 把(28*28)**256所有的情况都枚举出来
- 然后写分支判断语句，判断每种情况是哪种分类
- 但这种情况是不行的，因为(28*28)**256是一个天文数字，写不出来这样的代码
### 机器学习
- 提取特征：比如说边缘、角点、纹理、统计特征等
- 然后使用机器学习算法，比如SVM/KNN
- 最后就能比较好的将图片分类
### 深度学习
- 将图片数据直接喂给深度学习算法，比如神经网络
- 通过反向传播机制，让算法自己找到最好的假设空间
- 最后能比较好的进行图片分类

## 均方误差
- 使用公式表达均方误差：$ E = 1/2 \sum_k (yk - tk)^2$

In [2]:
import numpy as np
def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)
# 使用softmax预测出来的一组数字的概率
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
# 使用one-hot编码的标签，这个样本的真实标签是2
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
# 计算示例中的均方误差
# 也就说这个样本所产生的均方误差大概是0.1
print(mean_squared_error(np.array(y), np.array(t)))

# 再来计算一个样本的均方误差
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
# 假设真实标签还是2
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
# 由于预测出来是7的概率最大，所以这个样本的损失还是比较大的
print(mean_squared_error(np.array(y), np.array(t)))

0.09750000000000003


## 交叉熵误差
- 使用公式表达交叉熵误差：$ E = - \sum_k tk \log(yk) $
- 相对于均方误差，这个误差只考虑真实标签那个预测情况，预测概率越接近于1，误差越小，反之越大

In [7]:
# 以上面mnist的结果，计算一下交叉熵的损失
def cross_entropy_error(y, t):
    # 为了防止log(0)对结果溢出，加上一个比较小的量，根据softmax的例子
    # 即使加了这delta也不影响最终的概率分布
    delta = 1e-7
    return -np.sum(t * np.log(y+delta))
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
print(cross_entropy_error(np.array(y), np.array(t)))

# 预测比较失败的那个交叉熵损失是多少
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
print(cross_entropy_error(np.array(y), np.array(t)))

0.510825457099338
2.302584092994546


# mini-batch学习
- 机器学习的过程就是对损失函数学习的过程
- 前面介绍的损失只是对单个样本的损失，损失函数的求解是对所有样本的损失的求和
- 但是所有的样本又太大了，学习起来成本太高
- 所以训练的时候常常将一些样本捆绑起来，形成一个batch，然后对一个batch进行学习
- 一个batch的交叉熵损失函数可以用公式表达为：$ E = -1/N \sum_N \sum_k tk \log(yk) $
- 其中N是batch的大小，k是每个样本的索引，yk是预测的概率，tk是真实的类别

In [4]:
# 使用keras加载mnist数据集
import numpy as np
from keras.datasets import mnist
(x_train, t_train), (x_test, t_test) = mnist.load_data()
print(x_train.shape)
print(x_test.shape)
print(t_train.shape)
print(t_test.shape)
# 把特征空间拉平，并转换成[0, 1]区间的值，把28*28的图片变成一维的
# 把标签空间转换成one-hot编码
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255
t_train = np.eye(10)[t_train.astype('int32')]
t_test = np.eye(10)[t_test.astype('int32')]
print(x_train.shape)
print(x_test.shape)
print(t_train.shape)
print(t_test.shape)

(60000, 28, 28)
(10000, 28, 28)
(60000,)
(10000,)
(60000, 784)
(10000, 784)
(60000, 10)
(10000, 10)


In [11]:
# 随机从训练集中抽取一些数据，用于训练
import numpy as np
# 从[0, 60000)区间随机抽取10个数字
batch_mask = np.random.choice(x_train.shape[0], 10)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]
print(batch_mask)
print(x_batch.shape)
print(t_batch.shape)
print(type(x_batch))
print(type(t_batch))
print(x_batch)
print(t_batch)

[17226 16465 58713  9539 16037 39596 51073 36135 21423  3347]
(10, 784)
(10, 10)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]


In [9]:
# 一个batch的交叉熵损失函数定义
def cross_entropy_error(y, t):
    # 如果仅有一个样本，那么就给它增加一个维度，表示是单一样本
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size

In [14]:
# 单个样本的交叉熵损失
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
print(cross_entropy_error(np.array(y), np.array(t)))
# 多个样本的交叉熵损失
y = np.array(y*10)
print(y.shape)
print(cross_entropy_error(y, t_batch))

2.302584092994546
(100,)
91.00478626443056
